In [3]:
#First
#import tensorflow as tf
#import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()
import shutil

#Second
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D
from tensorflow.python.keras.layers.convolutional import MaxPooling2D
from tensorflow.python.keras.layers.core import Activation
from tensorflow.python.keras.layers.core import Flatten
from tensorflow.python.keras.layers.core import Dropout
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras import backend as K

#Third
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os


# import the necessary packages
from tensorflow.python.keras.models import load_model
import imutils

# Scripts to manipulate dataset

In [21]:
def split_test_into_subfolders(path):
    
    dest = 'sc5-Test-tensorflow'
    if not os.path.exists(dest):
        os.makedirs(dest)
    
    with open(path) as file:
        
        for line in file:
            file_name, class_name = line.split(';')
            class_name = class_name.replace('\n','')
            
            subdest = dest + '/' + class_name
            if not os.path.exists(subdest):
                os.makedirs(subdest)
            shutil.copy('sc5-Test/'+file_name, subdest)
            
        file.close()

In [25]:
def split_test_into_family_class(path):
    
    dest = 'sc5-test-tensorflow'
    start = 'sc5-Test-tensorflow-tmp'
    if not os.path.exists(dest):
        os.makedirs(dest)
        
    with open(path) as file:
        
        for line in file:
            file_name, class_name = line.split(';')
            class_name = class_name.replace('\n','')
            
            if os.path.exists(start+'/'+file_name):
                subdest = dest + '/' + class_name
                
                if not os.path.exists(subdest):
                    os.makedirs(subdest)
                shutil.move(start+'/'+file_name,subdest)
        file.close()  

In [18]:
def split_into_family_class(path):
    
    dest = 'sc5-tensorflow'
    if not os.path.exists(dest):
        os.makedirs(dest)
        
    with open(path) as file:
        
        for line in file:
            file_name, class_name = line.split(';')
            class_name = class_name.replace('\n','')
            
            if os.path.exists('sc5/'+file_name):
                subdest = dest + '/' + class_name
                
                if not os.path.exists(subdest):
                    os.makedirs(subdest)
                shutil.move('sc5/'+file_name,subdest)
        file.close()  

In [9]:
def balance_train_and_test(source_train, source_test):
    
    for family in os.listdir(source_train):
        
        for classes in os.listdir(source_train+'/'+family):
            
            subfolder_train = source_train + '/' + family + '/' + classes
            subfolder_test = source_test + '/' + family + '/' + classes
            
            if not os.path.exists(subfolder_test):
                print(classes)
                os.mkdir(subfolder_test)
                pics_to_move = int(0.2*len(os.listdir(source_train + '/' + family + '/' + classes)))
                images = os.listdir(subfolder_train)
                
                for i in range(pics_to_move):
                    image = subfolder_train + '/' + images[i]
                    shutil.move(image, subfolder_test)

In [10]:
balance_train_and_test('sc5-tensorflow','sc5-test-tensorflow')

VigilidelFuoco
Lanciamaggioredi10mMarrone
Caorlina
Sanpierota
Cacciapesca


In [8]:
split_into_subfolders('sc5-Test/ground_truth.txt')

In [19]:
split_into_family_class('sc5/ground-truth-family.txt')

In [27]:
split_test_into_family_class('sc5-Test/ground-truth-family.txt')

# Preprocessing CNN

In [20]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 40
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3) #Height x Width x RGB

In [5]:
def read_images(path, depth):
    # initialize the data and labels
    data = []
    labels = []

    # grab the image paths and randomly shuffle them
    print("[INFO] loading images...")
    imagePaths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(imagePaths)

    # loop over the input images
    with tf.device("/GPU:0"):
        for imagePath in imagePaths:
            # load the image, pre-process it, and store it in the data list
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
            image = img_to_array(image)
            data.append(image)

            # extract the class label from the image path and update the
            # labels list
            label = imagePath.split(os.path.sep)[-depth]
            labels.append(label)
    return data, labels

In [6]:
#depth:
# -1 image directory
# -2 boats classification
# -3 family classification
# -4 source path
data, labels = read_images('sc5-tensorflow', 2)

[INFO] loading images...


In [8]:
class SmallerVGGNet:
    
    #width : The image width dimension.
    #height : The image height dimension.
    #depth : The depth of the image — also known as the number of channels.
    #classes : The number of classes in our dataset 
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = keras.Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
 
        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
 
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
 
        # return the constructed network architecture
        return model

In [9]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
    data.nbytes / (1024 * 1000.0)))
 
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
 
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.2, random_state=42)


# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")


[INFO] data matrix: 1029.02MB


In [10]:
print(trainX.dtype, trainY.dtype, testX.dtype, testY.dtype)

float64 int64 float64 int64


# Training model

In [24]:
# initialize the model
with tf.device("/GPU:0"):
    print("[INFO] compiling model...")
    model = SmallerVGGNet.build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
        depth=IMAGE_DIMS[2], classes=labels.shape[1])

    #opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
    #opt = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    opt = tf.train.AdamOptimizer(learning_rate=INIT_LR)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    # train the network
    print("[INFO] training network...")
    H = model.fit_generator(
        aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY),
        steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS, verbose=1)

[INFO] compiling model...
[INFO] training network...
Epoch 1/40


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_25/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_5/TFOptimizer/gradients/conv2d_25/Conv2D_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_25/Conv2D/ReadVariableOp)]]
	 [[{{node metrics_5/acc/Mean/_1523}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1926_metrics_5/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [50]:
# save the model to disk
print("[INFO] serializing network...")
model.save('/')
 
# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open('/', "wb")
f.write(pickle.dumps(lb))
f.close()

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(args["plot"])

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17192628173080568651, name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17202135260280330295
 physical_device_desc: "device: XLA_GPU device", name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8783410648667573867
 physical_device_desc: "device: XLA_CPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1464860672
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12522700150487237094
 physical_device_desc: "device: 0, name: GeForce 940M, pci bus id: 0000:0a:00.0, compute capability: 5.0"]

# Testing model

In [2]:
# load the image
imagePaths = sorted(list(paths.list_images(path)))

 
# pre-process the image for classification
#image = cv2.resize(image, (96, 96))
#image = image.astype("float") / 255.0
#image = img_to_array(image)
#image = np.expand_dims(image, axis=0)

for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    data.append(image)
    
    image = np.expand_dims(image, axis=0)
    # extract the class label from the image path and update the
    # labels list
    #label = imagePath.split(os.path.sep)[-depth]
    #labels.append(label)
output = image.copy()

# load the trained convolutional neural network and the label
# binarizer
print("[INFO] loading network...")
model = load_model(args["model"])
lb = pickle.loads(open(args["labelbin"], "rb").read())
 
# classify the input image
print("[INFO] classifying image...")
proba = model.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

NameError: name 'path' is not defined

In [ ]:
# we'll mark our prediction as "correct" of the input image filename
# contains the predicted label text (obviously this makes the
# assumption that you have named your testing image files this way)
filename = args["image"][args["image"].rfind(os.path.sep) + 1:]
correct = "correct" if filename.rfind(label) != -1 else "incorrect"
 
# build the label and draw the label on the image
label = "{}: {:.2f}% ({})".format(label, proba[idx] * 100, correct)
output = imutils.resize(output, width=400)
cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
	0.7, (0, 255, 0), 2)
 
# show the output image
print("[INFO] {}".format(label))
cv2.imshow("Output", output)
cv2.waitKey(0)

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

ImportError: Traceback (most recent call last):
  File "/home/sivio/.local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/sivio/.local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/sivio/.local/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/e